In [10]:
def read_pdb(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

def parse_models(lines):
    models = []
    current_model = []
    for line in lines:
        if line.startswith("MODEL"):
            current_model = []
        elif line.startswith("ENDMDL"):
            models.append(current_model)
        elif line.startswith("ATOM"):
            current_model.append(line)
    return models

def extract_coordinates(models):
    coordinates = []
    for model in models:
        model_coords = []
        for line in model:
            if line.startswith("ATOM"):
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                model_coords.append((x, y, z))
        coordinates.append(model_coords)
    return coordinates

def average_coordinates(coordinates):
    import numpy as np
    num_atoms = len(coordinates[0])
    num_models = len(coordinates)
    averaged_coords = []
   
    for i in range(num_atoms):
        atom_coords = np.array([model[i] for model in coordinates])
        avg_x = np.mean(atom_coords[:, 0])
        avg_y = np.mean(atom_coords[:, 1])
        avg_z = np.mean(atom_coords[:, 2])
        averaged_coords.append((avg_x, avg_y, avg_z))
       
    return averaged_coords

def write_averaged_pdb(lines, averaged_coords, output_path):
    with open(output_path, 'w') as file:
        atom_index = 0
        for line in lines:
            if line.startswith("ATOM"):
                if atom_index < len(averaged_coords):
                    new_line = line[:30] + f"{averaged_coords[atom_index][0]:8.3f}" + \
                               f"{averaged_coords[atom_index][1]:8.3f}" + \
                               f"{averaged_coords[atom_index][2]:8.3f}" + line[54:]
                    file.write(new_line)
                    atom_index += 1
                else:
                    # Skip any extra ATOM lines that do not have corresponding averaged coordinates
                    continue
            elif line.startswith("MODEL") or line.startswith("ENDMDL"):
                continue
            else:
                file.write(line)

# Main execution
file_path = "Downloads/2lyf.pdb"  # Path to the input PDB file
output_path = "Downloads/real_2lyf.pdb"  # Path to the output PDB file

# Read and parse the PDB file
lines = read_pdb(file_path)
models = parse_models(lines)

# Ensure the number of models is as expected
num_models = len(models)
print(f"Number of models: {num_models}")

# Extract coordinates and average them
coordinates = extract_coordinates(models)

# Ensure that all models have the same number of atoms
num_atoms_per_model = [len(model) for model in coordinates]
print(f"Number of atoms per model: {num_atoms_per_model}")

if len(set(num_atoms_per_model)) != 1:
    raise ValueError("Not all models have the same number of atoms")

averaged_coords = average_coordinates(coordinates)

# Write the averaged coordinates to a new PDB file
write_averaged_pdb(lines, averaged_coords, output_path)

print(f"Averaged PDB file written to {output_path}")

Number of models: 20
Number of atoms per model: [282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282, 282]
Averaged PDB file written to Downloads/real_2lyf.pdb
